# DCGAN Implementation for MNIST and Fashion-MNIST

This code implements a **Deep Convolutional Generative Adversarial Network (DCGAN)** to generate synthetic images for both the **MNIST (handwritten digits)** and **Fashion-MNIST (clothing items)** datasets. Let's break down what's happening in this implementation:

---

##  Data Preparation

- Creates an output directory called `gan_outputs` to store generated images.
- Loads both the MNIST and Fashion-MNIST datasets using TensorFlow's built-in functionality.
- A preprocessing function:
  - Normalizes pixel values to the range [-1, 1].
  - Adds a channel dimension (needed for convolutional layers).
- Class ID mapping:
  - **Fashion-MNIST**: Focuses on 5 classes: `t-shirt`, `trouser`, `sandal`, `shirt`, `sneaker`.
  - **MNIST**: Focuses on 5 digits: `0`, `1`, `2`, `3`, `7`.

---

##  GAN Architecture

This implementation follows the standard **DCGAN** structure:

###  Generator Network

- **Input**: 100-dimensional noise vector.
- **Layers**:
  - Dense → reshape to (7×7×256)
  - BatchNorm + LeakyReLU
  - TransposedConv to (7×7×128)
  - TransposedConv to (14×14×64)
  - TransposedConv to (28×28×1)
- **Output**: Uses `tanh` activation to match normalized [-1, 1] image values.

###  Discriminator Network

- **Input**: 28×28×1 image (real or generated)
- **Layers**:
  - Conv2D with strides → (14×14×64)
  - Conv2D with strides → (7×7×128)
  - LeakyReLU + Dropout (30%)
  - Flatten + Dense → outputs real/fake probability

---
##  Loss Functions and Training

- **Loss Function**: Binary cross-entropy
  - Discriminator: classify real images as `1`, fake as `0`
  - Generator: tries to trick discriminator (fake as `1`)
- **Optimizer**: Adam (learning rate = 1e-4)
- **Training Loop**:
  - Custom training loop with `tf.GradientTape`
  - Decorated with `@tf.function` for performance

---

##  Image Generation Process

- **Separate GANs** are trained for each selected class in MNIST and Fashion-MNIST.

### For Each Class:

1. Select up to 1,000 training examples of that class.
2. Save one **real** example for reference.
3. Train a class-specific GAN for **1,000 epochs**.
4. Generate **9 synthetic images**:
   - One saved as `gan_dataset_class.png`
   - Eight saved as `aug_dataset_class_N.png`

---

## Key Technical Aspects

###  Class-Conditional Generation
- Trains a **separate GAN for each class**, which helps improve generation quality.

###  Architecture Choices
- Follows DCGAN best practices:
  - **Strided convolutions** instead of pooling.
  - **Batch normalization** in both networks (except outputs).
  - **LeakyReLU** for stable training.
  - **No hidden dense layers** in the middle.

###  Visualization
- Automatically saves **real** and **generated** samples to help assess performance.

---

##  Why This Matters

This implementation demonstrates how **GANs** can be used for **data augmentation**:

- Generates new, realistic samples for each class.
- Can be useful in training more robust classifiers.
- Preserves distributional properties of real data while introducing novel variations.


In [1]:

# DCGAN Trainer for FashionMNIST and MNIST (Class-Specific Samples)

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
import os

# Create output folder
output_dir = "gan_outputs"
os.makedirs(output_dir, exist_ok=True)

# Load datasets
(fashion_x_train, fashion_y_train), _ = tf.keras.datasets.fashion_mnist.load_data()
(mnist_x_train, mnist_y_train), _ = tf.keras.datasets.mnist.load_data()

# Normalize and reshape
def preprocess(x):
    x = (x.astype('float32') - 127.5) / 127.5
    return np.expand_dims(x, axis=-1)

fashion_x_train = preprocess(fashion_x_train)
mnist_x_train = preprocess(mnist_x_train)

# Class labels to generate
fashion_labels = {
    0: 'tshirt',
    1: 'trouser',
    5: 'sandal',
    6: 'shirt',
    7: 'sneaker'
}

mnist_labels = {
    0: '0',
    1: '1',
    2: '2',
    3: '3',
    7: '7'
}

# DCGAN Components
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(7*7*256, use_bias=False, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((7, 7, 256)),
        layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    return cross_entropy(tf.ones_like(real_output), real_output) + cross_entropy(tf.zeros_like(fake_output), fake_output)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Training loop
def train_dcgan(data, label_name, dataset_name, epochs=1000, batch_size=256):
    generator = build_generator()
    discriminator = build_discriminator()
    generator_optimizer = tf.keras.optimizers.Adam(1e-4)
    discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

    dataset = tf.data.Dataset.from_tensor_slices(data).shuffle(10000).batch(batch_size)

    @tf.function
    def train_step(images):
        noise = tf.random.normal([batch_size, 100])
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = generator(noise, training=True)
            real_output = discriminator(images, training=True)
            fake_output = discriminator(generated_images, training=True)
            gen_loss = generator_loss(fake_output)
            disc_loss = discriminator_loss(real_output, fake_output)
        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
        return gen_loss, disc_loss

    print(f"Training DCGAN for {dataset_name} class: {label_name}")
    for epoch in range(epochs):
        for image_batch in dataset:
            g_loss, d_loss = train_step(image_batch)
        if epoch % 100 == 0 or epoch == epochs - 1:
            print(f"Epoch {epoch}: Gen Loss = {g_loss.numpy():.4f}, Disc Loss = {d_loss.numpy():.4f}")

    # Save images
    noise = tf.random.normal([9, 100])
    generated_images = generator(noise, training=False).numpy()
    for i, img in enumerate(generated_images):
        img = (img * 127.5 + 127.5).astype('uint8').squeeze()
        fname = f"gan_{dataset_name}_{label_name}.png" if i == 0 else f"aug_{dataset_name}_{label_name}_{i}.png"
        plt.imsave(os.path.join(output_dir, fname), img, cmap='gray')

# Generate for FashionMNIST
for class_id in fashion_labels:
    label = fashion_labels[class_id]
    idx = np.where(fashion_y_train == class_id)[0][:1000]
    samples = fashion_x_train[idx]
    real_sample = (samples[0] * 127.5 + 127.5).astype('uint8').squeeze()
    plt.imsave(f"{output_dir}/real_fashion_{label}.png", real_sample, cmap='gray')
    train_dcgan(samples, label, "fashion")

# Generate for MNIST
for class_id in mnist_labels:
    label = mnist_labels[class_id]
    idx = np.where(mnist_y_train == class_id)[0][:1000]
    samples = mnist_x_train[idx]
    real_sample = (samples[0] * 127.5 + 127.5).astype('uint8').squeeze()
    plt.imsave(f"{output_dir}/real_mnist_{label}.png", real_sample, cmap='gray')
    train_dcgan(samples, label, "mnist")

print("Done generating real and GAN-based images.")


C:\Users\sathu\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\sathu\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training DCGAN for fashion class: tshirt
Epoch 0: Gen Loss = 0.6530, Disc Loss = 1.3437
Epoch 100: Gen Loss = 0.7959, Disc Loss = 1.3327
Epoch 200: Gen Loss = 0.6535, Disc Loss = 1.3906
Epoch 300: Gen Loss = 0.6940, Disc Loss = 1.4324
Epoch 400: Gen Loss = 0.7208, Disc Loss = 1.3440
Epoch 500: Gen Loss = 0.7477, Disc Loss = 1.5819
Epoch 600: Gen Loss = 0.9946, Disc Loss = 1.1122
Epoch 700: Gen Loss = 0.7924, Disc Loss = 1.3116
Epoch 800: Gen Loss = 1.2244, Disc Loss = 0.9063
Epoch 900: Gen Loss = 0.9347, Disc Loss = 1.1511
Epoch 999: Gen Loss = 0.9900, Disc Loss = 1.0188
Training DCGAN for fashion class: trouser
Epoch 0: Gen Loss = 0.5961, Disc Loss = 1.2615
Epoch 100: Gen Loss = 0.6993, Disc Loss = 1.4320
Epoch 200: Gen Loss = 0.7269, Disc Loss = 1.3393
Epoch 300: Gen Loss = 0.7089, Disc Loss = 1.4335
Epoch 400: Gen Loss = 0.6766, Disc Loss = 1.4937
Epoch 500: Gen Loss = 0.7345, Disc Loss = 1.4377
Epoch 600: Gen Loss = 0.7330, Disc Loss = 1.4266
Epoch 700: Gen Loss = 0.9141, Disc Loss

KeyboardInterrupt: 